In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../tutorials/fastai/old/") # go to parent dir

from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestClassifier
from IPython.display import display

from sklearn import metrics
from sklearn.metrics import f1_score


Bad key "text.kerning_factor" on line 4 in
/home/tom/anaconda3/envs/fastai/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
PATH = "datasets/base"

In [4]:
!ls {PATH}

pyhs-raw  pyhs-raw-corrected


In [5]:
lead_1 = pd.read_feather('datasets/base/pyhs-raw')
lead_1

,age,sex,fmax,mean_RR,mean_R_Peaks,mean_T_Peaks,mean_P_Peaks,mean_Q_Peaks,mean_S_Peaks,median_RR,...,var_RR,var_R_Peaks,skew_RR,skew_R_Peaks,kurt_RR,kurt_R_Peaks,mean_P_Onsets,mean_T_Offsets,HRV,label
0,64,1,8.203281,4460.777778,1.906904e+05,84.166667,77.666667,-32.750000,-46.666667,4017.0,...,8.233628e+06,3.074513e+10,0.326896,0.452789,-1.114599,-1.285166,4617.666667,5106.166667,300.261749,PAC\n
1,24,1,2.150430,9963.953488,1.612913e+05,183.904762,44.047619,-38.190476,-212.047619,9914.0,...,3.436726e+07,1.481045e+10,0.008229,-0.012282,-1.196234,-1.955239,10309.619048,10776.285714,252.051913,Normal\n
2,59,0,1.054852,7031.666667,2.289459e+05,120.466667,58.733333,-36.200000,-130.266667,6993.0,...,1.704473e+07,4.754209e+10,0.016040,0.102263,-1.178660,-1.859521,6736.266667,7255.600000,247.390310,PAC\n
3,14,1,7.645041,6695.090909,4.049139e+06,56.714286,57.190476,-148.619048,-121.952381,6660.0,...,1.752821e+07,8.191544e+11,0.031014,-4.325836,-1.220443,16.835113,6795.238095,7248.285714,319.844531,Normal\n
4,40,1,2.182612,5615.250000,1.302676e+06,196.000000,40.818182,-218.636364,-259.454545,5640.0,...,1.072786e+07,1.682837e+12,-0.014445,0.045967,-1.216803,-1.940233,5709.272727,6233.818182,237.254954,RBBB\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6872,61,1,2.200880,5355.391304,6.515502e+05,4.000000,4.600000,-13.800000,-558.100000,5212.0,...,8.467770e+06,5.689022e+11,-0.042738,0.273157,-1.121451,-1.911613,5066.800000,5409.800000,231.441628,AF
6873,78,1,4.201681,5052.095238,2.516323e+06,33.583333,-16.833333,-116.583333,-231.250000,4936.0,...,7.846417e+06,5.001414e+12,-0.059236,-0.246734,-0.964861,-1.885106,4722.000000,5218.000000,254.732016,AF\n
6874,12,1,1.583861,6097.157895,3.228570e+05,276.333333,30.777778,-94.222222,-266.166667,6134.0,...,1.271993e+07,5.643145e+10,-0.021902,0.088967,-1.201353,-1.860108,6067.111111,6543.000000,160.942494,Normal\n
6875,59,1,6.102441,5006.153846,7.516309e+05,-15.900000,13.333333,-36.500000,-34.700000,5632.0,...,9.243060e+06,3.005175e+11,-0.216776,-0.494394,-1.168932,-1.662923,NaN,5240.400000,413.754658,I-AVB\n


In [6]:
lead_2 = pd.read_feather('datasets/multilead/pyhs-raw-lead2')
lead_2

,age,sex,fmax,mean_RR,mean_R_Peaks,mean_T_Peaks,mean_P_Peaks,mean_Q_Peaks,mean_S_Peaks,median_RR,...,var_RR,var_R_Peaks,skew_RR,skew_R_Peaks,kurt_RR,kurt_R_Peaks,mean_P_Onsets,mean_T_Offsets,HRV,label
0,64,1,4.601841,5164.823529,3.214186e+06,115.636364,13.181818,-114.363636,-54.090909,5166.0,...,9.476354e+06,6.085160e+12,0.002240,-0.497951,-1.256533,-1.593309,5029.090909,5508.909091,324.786083,PAC\n
1,24,1,1.050210,9965.069767,1.423544e+06,371.350000,-19.571429,-114.714286,-270.809524,9920.0,...,3.436797e+07,1.638203e+12,0.008188,-0.033648,-1.196573,-1.982349,10306.761905,NaN,250.689810,Normal\n
2,59,0,1.054852,7262.193548,7.820505e+05,172.266667,-8.733333,-95.933333,-39.066667,7150.0,...,1.810798e+07,6.280158e+11,0.008420,0.119925,-1.192760,-1.895794,6725.600000,7256.933333,244.448222,PAC\n
3,14,1,4.572735,6977.000000,5.447730e+05,108.761905,44.047619,-72.476190,-157.095238,7275.0,...,1.720361e+07,4.369731e+10,-0.099483,-2.120175,-1.213845,2.765965,6830.095238,7244.285714,297.769315,Normal\n
4,40,1,8.548563,4981.000000,1.035438e+06,25.818182,95.272727,-62.363636,-360.363636,4715.0,...,9.543899e+06,7.479752e+11,0.134001,-0.406748,-1.146664,-1.832525,5675.818182,6216.545455,286.387022,RBBB\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6872,61,1,2.200880,4668.666667,1.825312e+05,31.083333,39.416667,-50.833333,-249.750000,4152.0,...,9.642657e+06,3.185242e+10,0.204868,0.142266,-1.288943,-1.781381,4792.000000,5256.666667,259.019304,AF
6873,78,1,1.200480,4809.684211,5.056835e+05,5.083333,5.250000,-2.333333,-121.583333,4682.0,...,8.069375e+06,1.562389e+11,-0.024755,-0.474755,-1.099878,-1.683698,4725.333333,5228.833333,277.583421,AF\n
6874,12,1,1.583861,6130.918919,1.151930e+06,251.944444,9.500000,-132.388889,-252.000000,6274.0,...,1.303593e+07,1.133624e+12,-0.045750,-0.039627,-1.238796,-1.978701,6113.444444,6549.333333,167.780114,Normal\n
6875,59,1,2.100840,4740.000000,7.664817e+05,22.800000,35.800000,-39.900000,-63.100000,5149.0,...,9.696095e+06,2.589422e+11,-0.072559,0.008766,-1.329275,-0.491303,4877.200000,5411.000000,390.501551,I-AVB\n


In [7]:
for df_raw in [lead_1, lead_2]:
    df_raw['label'][df_raw.loc[:, 'label'] == 'AF\n'] = 'AF'
    df_raw['label'][df_raw.loc[:, 'label'] == 'I-AVB\n'] = 'I-AVB'
    df_raw['label'][df_raw.loc[:, 'label'] == 'LBBB\n'] = 'LBBB'
    df_raw['label'][df_raw.loc[:, 'label'] == 'Normal\n'] = 'Normal'
    df_raw['label'][df_raw.loc[:, 'label'] == 'PAC\n'] = 'PAC'
    df_raw['label'][df_raw.loc[:, 'label'] == 'PVC\n'] = 'PVC'
    df_raw['label'][df_raw.loc[:, 'label'] == 'RBBB\n'] = 'RBBB'
    df_raw['label'][df_raw.loc[:, 'label'] == 'STD\n'] = 'STD'
    df_raw['label'][df_raw.loc[:, 'label'] == 'STE\n'] = 'STE'

/home/tom/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/tom/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/tom/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [8]:
labels = pd.get_dummies(lead_1['label'])
labels.describe()

,AF,I-AVB,LBBB,Normal,PAC,PVC,RBBB,STD,STE
count,6877.000000,6877.000000,6877.000000,6877.000000,6877.000000,6877.000000,6877.000000,6877.000000,6877.000000
mean,0.177548,0.104988,0.028937,0.133488,0.079104,0.091173,0.243566,0.114294,0.026901
std,0.382160,0.306560,0.167642,0.340127,0.269921,0.287877,0.429265,0.318191,0.161807
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
labels = pd.get_dummies(lead_2['label'])
labels.describe()

,AF,I-AVB,LBBB,Normal,PAC,PVC,RBBB,STD,STE
count,6877.000000,6877.000000,6877.000000,6877.000000,6877.000000,6877.000000,6877.000000,6877.000000,6877.000000
mean,0.177548,0.104988,0.028937,0.133488,0.079104,0.091173,0.243566,0.114294,0.026901
std,0.382160,0.306560,0.167642,0.340127,0.269921,0.287877,0.429265,0.318191,0.161807
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
lead_1.to_feather('datasets/base/pyhs-raw-corrected')
lead_2.to_feather('datasets/multilead/pyhs-raw-lead2-corrected')